In [77]:
#This program ask for coordinates of Brugge downtown and return a 3D plot of the LIDAR DSM map
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LightSource
import numpy as np
import rasterio as rs
from rasterio.mask import mask
from shapely.geometry import Polygon
import pyproj
import re
import ipywidgets as widgets
from IPython.display import display
from scipy.interpolate import griddata  








def main():
    
    html = widgets.HTML(
    value='<b style="font-size:20px">Accepted Coordinates Types (Latitude, Longitude), Location Brugge downtown</b>',
    description=''
)
    display(html)
    
    html2 = widgets.HTML(
    value='''<ul><li><i style="font-size:15px"><b>DMS :</b> ex ; 51°12'33.653"N, 3°13"28.918"E </i></li></ul>''',
    description=''
        )
    display(html2)
    
    
    html3 = widgets.HTML(
    value='''<ul><li><i style="font-size:15px"><b>WGS84 :</b> ex ; 51.209348, 3.2246995</i></li></ul>''',
    description=''
        )
    display(html3)
    
    html4 = widgets.HTML(
    value='''<ul><li><i style="font-size:15px"><b>LAMBERT72 :</b> ex ; 211658.26, 70053.52</i></li></ul>''',
    description=''
        )
    display(html4)
    
    
        
    
    
    @widgets.interact_manual(Coodrinates='', 
        BoundingBox =(10,200,5), Color =['viridis', 'plasma', 'YlGn', 'cubehelix'] )
    
    
    def plot(Coordinates='',BoundingBox = 40, Color = 'viridis', grid=False):
    #asking for input
        

    #function that transform DMS coordinates to DD WSG84 coordinates
        def dms2dd():
            #Extraction of longitude / latitude
            LON, LAT = map(str, Coordinates.split(''','''))

            #extraction of figure with regex
            degreesLON, minutesLON, secondsLON, directionLON = re.split('[°\'"]+', LON)
            #Transfomation formula
            ddLON = float(degreesLON) + float(minutesLON) / 60 + float(secondsLON) / (60 * 60);
            #Negative transformation for west values
            if directionLON in ('W'):
                ddLON *= -1

            # extraction of figure with regex
            degreesLAT, minutesLAT, secondsLAT, directionLAT = re.split('[°\'"]+', LAT)
            # Transfomation formula
            ddLAT = float(degreesLAT) + float(minutesLAT) / 60 + float(secondsLAT) / (60 * 60);
            # Negative transformation for south values
            if directionLAT in ('S'):
                ddLAT *= -1
            #return of the result
            return (ddLAT, ddLON)

    #If there is E in the input, apply the function
        if 'E' in Coordinates:
            x1, y1 = dms2dd()
    #Else separate the float values
        
        else:
            y1, x1 = map(float, Coordinates.split(''','''))

        #If the float are below 180° we are in WGS84 otherwise in Lambert72
        if x1 < 180 and y1 < 180:
            proj = pyproj.Transformer.from_crs(4326, 31370)
            x1, y1 = proj.transform(y1, x1)

   
    # creation of the region of interest (roi) with polygon, a square
        roi = Polygon([(x1 - int(BoundingBox / 2), y1 + int(BoundingBox / 2)),
                   (x1 + int(BoundingBox / 2), y1 + int(BoundingBox / 2)),
                   (x1 + int(BoundingBox / 2), y1 - int(BoundingBox / 2)),
                   (x1 - int(BoundingBox / 2), y1 - int(BoundingBox / 2))])

    # Opening of the file
        fp = r'D:\GitHub\Data_Tools_Final_Challenge_EM\DHMVIIDSMRAS1m_k13(1)\GeoTIFF\DHMVIIDSMRAS1m_k13.tif'
        with rs.open(fp) as src:
        # Masking of the image
            out_image, out_transform = rs.mask.mask(src, shapes=[roi], crop=True, filled=False)
            out_meta = src.meta.copy()
            out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
        
    
        # getting rid of the NAN value and change to numerical value
        diff_surf = np.nan_to_num(out_image)

        # Defining of x and y
        temp_x = np.arange(diff_surf.shape[2])
        temp_y = np.arange(diff_surf.shape[1])
        Z_diff = diff_surf[0]
        # Fliping of y value
        temp_y = np.flip(temp_y, 0)

        # Creation of the meshgrid
        #xnew, ynew = np.mgrid[temp_x:80j, temp_y:80j]
        # Definition of Z layer
        X_diff, Y_diff = np.meshgrid(temp_x, temp_y)
        #zi = griddata((temp_x.flatten(), temp_y.flatten()), Z_diff.flatten(), (X_diff, Y_diff), method='linear')
        #f = interpolate.interp2d(X_diff, Y_diff, Z_diff, kind = 'cubic')
        #znew = f(temp_x, temp_y)
        
        

        
        
        # Create light source object.
        #ls = LightSource(90, 10)
        # Shade data, creating an rgb array.
        #rgb = ls.shade(Z_diff, cmap =cm.viridis)
        # Plot
        # Plot DSM in Python
        
        fig_dsm = plt.figure(figsize=(12, 6))
        fig_dsm.canvas.toolbar_position = 'bottom'
        stride=1
        #ax = axes[1,1]
        # definition of the subplot
        ax = fig_dsm.add_subplot(1,2,1, projection='3d')
        # axis off
        ax.axis(grid)
        #ax.set_title('{0}x{0} data points, stride={1}'.format(N,stride))
        surf = ax.plot_surface(X_diff, Y_diff,Z_diff, rstride=stride, cstride=stride, linewidth=0.0, cmap=Color,  antialiased=False)
        ax.view_init(50, 250)
        # limit of the Z height and plot of title
        ax.set_zlim3d(0, 30)
        cbar = fig_dsm.colorbar(surf, shrink=0.6, aspect=8)
        cbar.set_label(label='Height [M]', size='large', weight='bold', fontsize = 10)
        
        
        ax = fig_dsm.add_subplot(1,2,2, projection='3d')
        # axis off
        ax.axis(grid)
        #ax.set_title('{0}x{0} data points, stride={1}'.format(N,stride))
        surf = ax.plot_surface(X_diff, Y_diff,Z_diff, rstride=stride, cstride=stride, linewidth=0.0, cmap=Color,  antialiased=False)
        ax.view_init(70, 220)
        # show the plot
        #plt.title("Digital Surface Model of your coorindates : " + Coordinates, 'left title')
        fig_dsm.tight_layout()
       
    

    
if __name__ == "__main__":
    main()


HTML(value='<b style="font-size:20px">Accepted Coordinates Types (Latitude, Longitude), Location Brugge downto…

HTML(value='<ul><li><i style="font-size:15px"><b>DMS :</b> ex ; 51°12\'33.653"N, 3°13"28.918"E </i></li></ul>'…

HTML(value='<ul><li><i style="font-size:15px"><b>WGS84 :</b> ex ; 51.209348, 3.2246995</i></li></ul>')

HTML(value='<ul><li><i style="font-size:15px"><b>LAMBERT72 :</b> ex ; 211658.26, 70053.52</i></li></ul>')

interactive(children=(Text(value='', description='Coordinates'), IntSlider(value=40, description='BoundingBox'…